In [110]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, DensityMatrix, Pauli, Operator, random_statevector, partial_trace
from qiskit_aer import AerSimulator, Aer
import numpy as np
from scipy.linalg import sqrtm
import cvxpy as cp
import itertools
from IPython.display import clear_output
from winsound import Beep

def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

def check_legal(matrix, print_errors=True):
    errors, legal = [], True
    if not hermitian(matrix):
        errors.append('not hermitian')
    if not trace_one(matrix):
        errors.append('trace not equal to one')
    if not positive_semi_definite(matrix):
        errors.append('not positive semidefinite')
    if len(errors) > 0:
        legal = False
    if print_errors:
        if not legal:
            print(f'input is not legal: ' + '; '.join(errors))
        else: 
            print('input is a legal density matrix')
    return legal
        
        
def generate_prob_lst(num_states):
    prob_lst = np.array([np.random.random() for _ in range(num_states)])
    prob_lst /= np.sum(prob_lst)
    return prob_lst

def get_rank(dm, tol=1e-10):
    return int(np.sum(np.linalg.eigvalsh(dm) > tol))

def get_fidelity(dm1, dm2, tol=1e-5):
    # assert is_legal(dm1) and is_legal(dm2), 'inputs are not legal density matrices'
    if not is_legal(dm1) and is_legal(dm2):
        print("Warning: inputs are not legal density matrices")
    try: 
        fidelity = (np.trace(sqrtm(sqrtm(dm1) @ dm2 @ sqrtm(dm1)))) ** 2
    except ValueError:
        print('fidelity cannot be computed for given inputs')
    assert np.abs(np.imag(fidelity)) < tol, 'fidelity is not real within tol'
    return fidelity.real

def generate_dm(num_qubits, num_states, state_lst=None, prob_lst=None, prime_prob=None):
    assert (prob_lst is None) or (prime_prob is None), 'cannot set prob_lst and prime_prob together'
    if state_lst is None:
        state_lst = [random_statevector(2**num_qubits) for _ in range(num_states)]
    if prime_prob is not None:
        prob_lst = np.array([prime_prob] + (generate_prob_lst(num_states - 1) * (1 - prime_prob)).tolist())
    elif prob_lst is None:
        prob_lst = generate_prob_lst(num_states)
    density_matrix = sum([DensityMatrix(state_lst[i]).data * prob_lst[i] for i in range(num_states)])
    return density_matrix

def generate_random_Pauli_strings(num_strings, length, contain_I=True):
    if contain_I:
        characters = ['I', 'X', 'Y', 'Z']
    else:
        characters = ['X', 'Y', 'Z']
    generated_strings = []
    assert num_strings < len(characters) ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != 'I' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_random_01_strings(num_strings, length):
    characters = ['0', '1']
    generated_strings = []
    assert num_strings < 2 ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != '0' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_uv_Pauli_matrix(u_vec, v_vec):
    uv_map = {
        '00':'I', '01':'Z', '10':'X', '11':'Y'
    }
    Pauli_string = ''
    for u, v in zip(u_vec, v_vec):
        Pauli_char = uv_map.get(str(u) + str(v))
        if Pauli_char is None:
            raise ValueError('u or v list contains elements neither 0 or 1')
        else:
            Pauli_string += Pauli_char
    return Pauli_string

def generate_all_binary(length):
    all_strings = itertools.product('01', repeat=length)
    result = [''.join(s) for s in all_strings if '1' in s]
    return result

def generate_Pauli_expectations(dm, obsv):
    return np.trace(dm @ Pauli(obsv).to_matrix()).real

def get_trace_norm(dm):
    return np.sum(np.linalg.svd(dm, compute_uv=False))

In [45]:
def estimate_Pauli_expectations(dm, obsv, num_samples, simulation=False):
    num_samples = int(num_samples)
    if simulation: # simulate the process of sampling
        exp = np.real(np.trace(dm @ Pauli(obsv).to_matrix()))
        prob_p1 = (1 + exp) / 2
        prob_m1 = 1 - prob_p1
        samples = np.random.choice([+1, -1], size=num_samples, p=[prob_p1, prob_m1])
        return np.mean(samples)
    else: # use the approximate distribution instead
        exp = np.real(np.trace(dm @ Pauli(obsv).to_matrix()))
        num_samples_root = num_samples ** .5
        std_dev = (1 - exp ** 2) ** .5 / num_samples_root
        return np.random.normal(exp, std_dev)

In [114]:
def get_partial_trace(dm, subsystems):
    return partial_trace(dm, subsystems).data

def get_von_neumann_entropy(dm, r=None):
    # if not np.allclose(dm, dm.conj().T):
    #     raise ValueError("The density matrix must be Hermitian.")
    if r is None:
        eigenvalues = np.linalg.eigvalsh(dm)
        eigenvalues = eigenvalues[eigenvalues > 0]
        entropy = - np.sum(eigenvalues * np.log2(eigenvalues))
    else:
        eigenvalues = np.linalg.eigvalsh(dm)
        eigenvalues = np.partition(eigenvalues, -r)[-r:]
        entropy = - np.sum(eigenvalues * np.log2(eigenvalues))
    return entropy

def get_TMI(dm):
    dm_s = get_partial_trace(dm, [1, 2])
    dm_m1 = get_partial_trace(dm, [0, 2])
    dm_m2 = get_partial_trace(dm, [1, 2])
    dm_m = get_partial_trace(dm, [0])
    dm_sm1 = get_partial_trace(dm, [2])
    dm_sm2 = get_partial_trace(dm, [1])
    i2_sm1 = get_von_neumann_entropy(dm_s) + get_von_neumann_entropy(dm_m1) - get_von_neumann_entropy(dm_sm1)
    i2_sm2 = get_von_neumann_entropy(dm_s) + get_von_neumann_entropy(dm_m2) - get_von_neumann_entropy(dm_sm2)
    i2_sm = get_von_neumann_entropy(dm_s) + get_von_neumann_entropy(dm_m) - get_von_neumann_entropy(dm)
    return i2_sm1 + i2_sm2 - i2_sm

In [55]:
# Prepare a state for measurement
num_qubit = 3
state = generate_dm(num_qubit, 5, prime_prob=.99)
check_legal(state)
r = np.sum(np.linalg.eigvals(state) > 2e-2)
d = 2 ** num_qubit
num_samples = 1e12

# Prepare observables
# c = 1
# len_u_lst = int(c * r * (np.log(d)) ** 2)
len_u_lst = 7 # manually set number of observables
v_lst = generate_all_binary(num_qubit)
u_lst = np.random.choice(v_lst, len_u_lst)
observables = [generate_uv_Pauli_matrix(u_vec, v_vec) for u_vec, v_vec in list(itertools.product(u_lst, v_lst))]
expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]

input is a legal density matrix


In [56]:
np.all(
    np.array(
        [np.abs(np.trace(state @ Pauli(o).to_matrix()) - e) <= 1e-5 for o, e in zip(observables, expectations)]
    )
)

np.True_

In [ ]:
def optimize(dim, obsv, expct, tol=1e-5):
    sigma = cp.Variable((dim, dim), complex=True)
    objective = cp.Minimize(cp.abs(5 * cp.norm(sigma, 'nuc') + 0 * cp.norm(sigma, 'fro') ** 2))
    constraints = [cp.trace(sigma) == 1]
    for o, e in zip(obsv, expct):
        constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    return sigma.value
    
    
sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state, tol=1e-6)}')
Beep(1000, 2000)

optimal
fidelity between outcome and target is 0.9983758114561067


In [93]:
print(get_TMI(state))
print(get_TMI(sigma))

-0.1523516230359947
-0.15004137995892886


In [100]:
errors = []

for _ in range(50):
    num_qubit = 3
    state = generate_dm(num_qubit, 5, prime_prob=.99)
    r = np.sum(np.linalg.eigvals(state) > 2e-2)
    d = 2 ** num_qubit
    num_samples = 1e12
    # len_u_lst = 7 # manually set number of observables
    # v_lst = generate_all_binary(num_qubit)
    # u_lst = np.random.choice(v_lst, len_u_lst)
    # observables = [generate_uv_Pauli_matrix(u_vec, v_vec) for u_vec, v_vec in list(itertools.product(u_lst, v_lst))]
    observables = generate_random_Pauli_strings(49, 3, contain_I=True)
    expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]
    def optimize(dim, obsv, expct, tol=1e-5):
        sigma = cp.Variable((dim, dim), complex=True)
        objective = cp.Minimize(cp.abs(5 * cp.norm(sigma, 'nuc') + 0 * cp.norm(sigma, 'fro') ** 2))
        constraints = [cp.trace(sigma) == 1]
        for o, e in zip(obsv, expct):
            constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
        problem = cp.Problem(objective, constraints)
        problem.solve()
        return sigma.value
    sigma = optimize(d, observables, expectations)
    error = (get_TMI(sigma) - get_TMI(state)) / get_TMI(state)
    errors.append(error)
    
print(f"the average error is {np.mean(errors)}")

the average error is -0.015146091048427986


In [115]:
errors, i3s = [], []

for _ in range(50):
    num_qubit = 3
    state = generate_dm(num_qubit, 5, prime_prob=.99)
    r = np.sum(np.linalg.eigvals(state) > 2e-2)
    d = 2 ** num_qubit
    num_samples = 1e12
    # len_u_lst = 7 # manually set number of observables
    # v_lst = generate_all_binary(num_qubit)
    # u_lst = np.random.choice(v_lst, len_u_lst)
    # observables = [generate_uv_Pauli_matrix(u_vec, v_vec) for u_vec, v_vec in list(itertools.product(u_lst, v_lst))]
    observables = generate_random_Pauli_strings(49, 3, contain_I=True)
    expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]
    def optimize(dim, obsv, expct, tol=1e-5):
        sigma = cp.Variable((dim, dim), complex=True)
        objective = cp.Minimize(cp.abs(5 * cp.norm(sigma, 'nuc') + 0 * cp.norm(sigma, 'fro') ** 2))
        constraints = [cp.trace(sigma) == 1]
        for o, e in zip(obsv, expct):
            constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
        problem = cp.Problem(objective, constraints)
        problem.solve()
        return sigma.value
    sigma = optimize(d, observables, expectations)
    error = (get_TMI(sigma) - get_TMI(state)) / get_TMI(state)
    errors.append(error)
    i3s.append(get_TMI(state))
    
print(f"the average error is {np.mean(errors)}")

the average error is 0.0037107325834547893


In [117]:
i3s

[np.float64(-0.0875529847131673),
 np.float64(-0.19666985491902889),
 np.float64(-0.10608325118141848),
 np.float64(0.06985814455450923),
 np.float64(-0.20072165453535185),
 np.float64(0.03422640469217342),
 np.float64(-0.0553975291336255),
 np.float64(-0.22015042830872256),
 np.float64(0.2812033900450295),
 np.float64(-0.009784107988202173),
 np.float64(-0.030401616923425134),
 np.float64(-0.14091500092749687),
 np.float64(-0.15012327987794394),
 np.float64(0.011135656304527952),
 np.float64(0.16762463464488353),
 np.float64(-0.176756743978836),
 np.float64(0.16096304659432525),
 np.float64(-0.0039635929147587845),
 np.float64(0.1863104107672542),
 np.float64(0.06483958144638957),
 np.float64(-0.23457551108653907),
 np.float64(-0.040617205639180476),
 np.float64(-0.2840706820950123),
 np.float64(-0.10278565934931394),
 np.float64(0.3140498545951027),
 np.float64(-0.012747693580562824),
 np.float64(0.038283533972016226),
 np.float64(-0.01188753974018808),
 np.float64(-0.167142973888356

In [116]:
print(get_TMI(state))
print(get_TMI(sigma))

0.020381276981997543
0.023802723160014416
